In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

import torchvision.transforms.functional as F
from torchvision.models.detection import keypointrcnn_resnet50_fpn, KeypointRCNN_ResNet50_FPN_Weights
from torchvision.utils import make_grid
from torchvision.io import read_image
from pathlib import Path

from torchvision.models.detection import maskrcnn_resnet50_fpn, MaskRCNN_ResNet50_FPN_Weights
from torchvision.utils import draw_segmentation_masks

from PIL import Image

import cv2
import numpy as np
from skimage import restoration, filters
from skimage.util import random_noise
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.io import imsave
import matplotlib.pyplot as plt

import os
import sys

%matplotlib inline

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


### Colocando a pasta raiz do projeto na pasta da imagem.

In [3]:
%cd ../../../TCC/

d:\Documentos\Faculdade - UFPI\Weight Estimation\predict_ovinos\TCC


In [4]:
# Define the helper function
def decode_segmap(image, nc=21):
  label_colors = np.array([(0, 0, 0),  # 0=background
                           # 1=aeroplane, 2=bicycle, 3=bird, 4=boat, 5=bottle
                           (128, 0, 0), (0, 128, 0), (128, 128,
                                                      0), (0, 0, 128), (128, 0, 128),
                           # 6=bus, 7=car, 8=cat, 9=chair, 10=cow
                           (0, 128, 128), (128, 128, 128), (64,
                                                            0, 0), (192, 0, 0), (64, 128, 0),
                           # 11=dining table, 12=dog, 13=horse, 14=motorbike, 15=person
                           (192, 128, 0), (64, 0, 128), (192, 0,
                                                         128), (64, 128, 128), (192, 128, 128),
                           # 16=potted plant, 17=sheep, 18=sofa, 19=train, 20=tv/monitor
                           (0, 64, 0), (128, 64, 0), (0, 192, 0), (128, 192, 0), (0, 64, 128)])
  r = np.zeros_like(image).astype(np.uint8)
  g = np.zeros_like(image).astype(np.uint8)
  b = np.zeros_like(image).astype(np.uint8)
  for l in range(0, nc):
    idx = image == l
    r[idx] = label_colors[l, 0]
    g[idx] = label_colors[l, 1]
    b[idx] = label_colors[l, 2]
  rgb = np.stack([r, g, b], axis=2)
  return rgb


In [5]:
from PIL import Image
import matplotlib.pyplot as plt
import torch
from torchvision import models
import torchvision.transforms as T
from skimage.color import rgb2gray
from skimage.io import imsave


def segment(net, path):
  img = Image.open(path)
  # plt.imshow(img)
  # plt.axis('off')
  # plt.show()

  # Comment the Resize and CenterCrop for better inference results
  trf = T.Compose([T.Resize(512),
                   T.ToTensor(),
                   T.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])])
  inp = trf(img).unsqueeze(0)
  out = net(inp)['out']
  om = torch.argmax(out.squeeze(), dim=0).detach().cpu().numpy()
  rgb = decode_segmap(om)
  rgb_gray = rgb2gray(rgb)
  # plt.imshow(rgb_gray, cmap="gray")
  # plt.axis('off')
  # plt.show()

  return rgb_gray


In [6]:
from torchvision.models.segmentation import fcn_resnet50, FCN_ResNet50_Weights, DeepLabV3_ResNet101_Weights

weights = DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1
dlab = models.segmentation.deeplabv3_resnet101(
    weights=weights, progress=False).eval()

In [7]:
imgs = list()
name_imgs = list()
path = "images_for_train/"
dirs = os.listdir(path)

for item in dirs:
    if os.path.isfile(path + item):
        print(path + item)
        imgs.append(segment(dlab, path + item))
        name_imgs.append(item)
        
        


images_for_train/r_amarelo.jpg
images_for_train/r_amarelo_15.jpg
images_for_train/r_animal_01.jpg
images_for_train/r_animal_02.jpg
images_for_train/r_animal_03.jpg
images_for_train/r_animal_04.jpg
images_for_train/r_animal_05.jpg
images_for_train/r_animal_06.jpg
images_for_train/r_animal_07.jpg
images_for_train/r_animal_08.jpg
images_for_train/r_animal_09.jpg
images_for_train/r_animal_10.jpg
images_for_train/r_animal_11.jpg
images_for_train/r_animal_12.jpg
images_for_train/r_animal_13.jpg
images_for_train/r_animal_14.jpg
images_for_train/r_animal_15.jpg
images_for_train/r_animal_16.jpg
images_for_train/r_animal_17.jpg
images_for_train/r_animal_18.jpg
images_for_train/r_animal_19.jpg
images_for_train/r_animal_20.jpg
images_for_train/r_animal_21.jpg
images_for_train/r_animal_22.jpg
images_for_train/r_animal_23.jpg
images_for_train/r_animal_24.jpg
images_for_train/r_animal_25.jpg
images_for_train/r_animal_26.jpg
images_for_train/r_animal_27.jpg
images_for_train/r_animal_28.jpg
images_for_

In [20]:
count = len(name_imgs)
folder = "D:\Documentos\Faculdade - UFPI\Weight Estimation\predict_ovinos\TCC\images_segmented_3"

for x in range(count):
  os.makedirs(folder, exist_ok=True)
  imsave("images_segmented_3/seg_image_" +
         str(name_imgs[x][:-4])+".png", imgs[x])


C:\Users\pedro\AppData\Local\Temp\ipykernel_15460\4021870383.py:6: UserWarning: images_segmented_3/seg_image_r_amarelo.png is a low contrast image
  imsave("images_segmented_3/seg_image_" +
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\pedro\AppData\Local\Temp\ipykernel_15460\4021870383.py:6: UserWarning: images_segmented_3/seg_image_r_animal_04.png is a low contrast image
  imsave("images_segmented_3/seg_image_" +
Lossy conversion from f